# Random Forest - Melbourne Housing

In [31]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
# Path of the file to read
iowa_file_path = 'melb_data.csv'

home_data = pd.read_csv(iowa_file_path)
home_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [32]:
home_data

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [33]:
def define_test_train_validation1(df, features):
    '''Splits the data set etc'''

    def missing_values():
        '''Displays missing values'''
        missing_val_count_by_column = (df.isnull().sum())
        missing_values = missing_val_count_by_column[missing_val_count_by_column > 0].apply(str) + f'/{df.shape[0]} missing'

        print(f'There are {df.shape[0]} rows and {df.shape[1]} columns.\n')
        print('Missing Values:\n')
        print(missing_values)

    def check_cardinality(df):
        # Categorical columns in the training data
        object_cols = [col for col in df.columns if df[col].dtype == "object"]
        # Columns that will be one-hot encoded
        low_cardinality_cols = [col for col in object_cols if df[col].nunique() < 10]

        # Columns that will be dropped from the dataset
        high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

        df = df.drop(high_cardinality_cols, axis=1)

    missing_values()
    #df = check_cardinality(df)

    # Target Objective
    y = df.Price
    # Features - fill in nans with mean, code string data


    X = pd.get_dummies(df[features]).apply(lambda x: x.fillna(x.mean()),axis=0)
    # Split Data
    train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
    return train_X, val_X, train_y, val_y


def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)

    print(f"Validation MAE for best value of {max_leaf_nodes} max leaf nodes: {mae:,.0f}")
    return mae


def find_best_tree_size(candidate_max_leaf_nodes):
    # Write loop to find the ideal tree size from candidate_max_leaf_nodes
    scores = {leaf_size:get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
    # Store the best value of max_leaf_nodes
    best_tree_size = min(scores, key=scores.get)
    return best_tree_size

def final_model(best_tree_size):
    final_model = RandomForestRegressor(max_leaf_nodes=best_tree_size, random_state=1)
    final_model.fit(train_X, train_y)
    final_model_predictions = final_model.predict(val_X)
    final_model_mae = mean_absolute_error(final_model_predictions, val_y)
    print(f"Validation MAE for Random Forest Model: {final_model_mae}")
    return final_model_predictions


features = ['Rooms', 'YearBuilt', 'Bathroom', 'Car', 'Landsize', 
            'BuildingArea', 'Lattitude', 'Longtitude', 'Postcode',
            'Method', 'SellerG'
            ]
train_X, val_X, train_y, val_y = define_test_train_validation1(home_data, features)

candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500, 1000]
#best_tree_size = find_best_tree_size(candidate_max_leaf_nodes)

#final_model_predictions = final_model(best_tree_size)

There are 13580 rows and 21 columns.

Missing Values:

Car               62/13580 missing
BuildingArea    6450/13580 missing
YearBuilt       5375/13580 missing
CouncilArea     1369/13580 missing
dtype: object


In [339]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost.sklearn import XGBRegressor
from xgboost import plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

mass_cols = ['pl_cmassj',
            'pl_cmassjerr1',
            'pl_cmassjerr2',
            'pl_cmassjlim',
            'pl_cmassjstr',
            'pl_cmasse',
            'pl_cmasseerr1',
            'pl_cmasseerr2',
            'pl_cmasselim',
            'pl_cmassestr',
            #'pl_massj',
            'pl_massjerr1',
            'pl_massjerr2',
            'pl_massjlim',
            'pl_massjstr',
            'pl_masse',
            'pl_masseerr1',
            'pl_masseerr2',
            'pl_masselim',
            'pl_massestr',
            'pl_bmassjerr1',
            'pl_bmassjerr2',
            'pl_bmassjlim',
            'pl_bmassjstr',
            'pl_bmasseerr1',
            'pl_bmasseerr2',
            'pl_bmasselim',
            'pl_bmassestr',
            'pl_bmassprov',
            'pl_msinij',
            'pl_msinijerr1',
            'pl_msinijerr2',
            'pl_msinijlim',
            'pl_msinijstr',
            'pl_msinie',
            'pl_msinieerr1',
            'pl_msinieerr2',
            'pl_msinielim',
            'pl_msiniestr',
            'pl_bmassj',
            'pl_bmasse',]

radius_cols = [#'pl_radj',
            'pl_radjerr1',
            'pl_radjerr2',
            'pl_radjlim',
            'pl_radjstr',
            'pl_rade',
            'pl_radeerr1',
            'pl_radeerr2',
            'pl_radelim',
            'pl_radestr',]



def read_data(filename, target):
    # Read the data
    X = pd.read_csv(filename,)#.drop(mass_cols, axis=1)

    # Remove rows with missing target, separate target from predictors
    X.dropna(axis=0, subset=[target, 'pl_radj'], inplace=True)
    y = X[target]
    # print(f'Target has {y.isnull().sum()} missing values.')
    X.drop([target], axis=1, inplace=True)

    # Break off validation set from training data
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            train_size=0.8, test_size=0.2,
                                            random_state=0)
    #print(X_train.columns)
    return X_train, X_test, y_train, y_test, X, y


def pipelines(hyper_paramters):
    '''
    Runs the preprocessing and model pipeline.
    '''

    # "Cardinality" means the number of unique values in a column
    # Select categorical columns with relatively low cardinality (convenient but arbitrary)
    categorical_cols = [cname for cname in X_train.select_dtypes('object').columns 
                        if X_train[cname].nunique() < 10] 

    # Select numerical columns
    numerical_cols = X_train.select_dtypes('number').columns 

    # Preprocessing for numerical data
    numerical_transformer =  Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('minmaxscaler', RobustScaler()),
        ])

    # Preprocessing for categorical data
    categorical_transformer = Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False)),
        ])

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Tune Params
    xgbRegressor_gridSearch = Pipeline(
        steps=[
            ('tune', GridSearchCV(
                XGBRegressor(random_state=0),
                hyper_parameters,
                cv = 2,
                n_jobs = 10,
                verbose=True,
                refit=True
                )
            )
        ])
    
    

    # Run the entire pipeline
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            #('model', XGBRegressor(random_state=0))
            ('model', xgbRegressor_gridSearch)
        ])

    # Preprocessing of training data, fit model 
    clf.fit(X_train, y_train)
    # Refit with the best params

    # Preprocessing of validation data, get predictions
    y_pred = clf.predict(X_test)
    # plot_tree(clf.named_steps['model'])
    # Save the model
    import pickle
    pickle.dump(clf, open('model.pkl', 'wb'))

    print(f'MAE: {mean_absolute_error(y_test, y_pred):.4f}')
    print(f'Accuracy: {clf.score(X_test, y_test) * 100:.2f}%')
    return clf, y_pred

# Parameter Tuning
hyper_parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
                    'objective':['count:poisson'],#['reg:squarederror'],
                    'learning_rate': [.04], #so called `eta` value
                    'max_depth': [3],
                    'min_child_weight': [0],
                    'subsample': [0.2],
                    'colsample_bytree': [1],
                    'n_estimators': [500]}

X_train, X_test, y_train, y_test, X, y = read_data('ML_nasa_tess_viable.csv.xz', 'pl_massj')
# X_train, X_test, y_train, y_test, X, y = read_data('nasa.csv', 'pl_massj')

clf, y_pred = pipelines(hyper_parameters)



/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (3,4,35,40,45,50,55,60,65,75,139,148,153,162,183,187,206,211,216,221,226,231,232,237,242,257,262) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Fitting 2 folds for each of 1 candidates, totalling 2 fits
MAE: 0.0580
Accuracy: 98.40%


In [244]:
# Prediction
y_pred = pd.Series(y_pred).rename('Prediction')
y_test = y_test.rename('Test')
# Set same indexes
y_pred.index = y_test.index

# Test
y_test_p = y_test.sample(10)
X_test_p = X_test['pl_radj'][y_test_p.index]
y_pred_p = y_pred[y_test_p.index]
X_test_p


255      1.310
7        1.440
3829     1.119
3084     0.267
7060     0.191
30938    1.240
5644     1.741
30928    1.570
30886    1.220
29900    1.378
Name: pl_radj, dtype: float64

In [341]:
def compare_predict_vs_test(X_test, y_test, y_pred, sample_size=20):
    '''Prints a table to compare the test versus the prediction.'''
    from tabulate import tabulate
    import mr_forecast as mr
    
    # Prediction
    y_pred = pd.Series(y_pred).rename('Prediction')
    y_test = y_test.rename('Test')

    # Set same indexes
    y_pred.index = y_test.index
    
    # Test
    y_test_p = y_test.sample(sample_size)
    X_test_p = X_test['pl_radj'][y_test_p.index]
    y_pred_p = y_pred[y_test_p.index]

    df = pd.DataFrame()
    df['Test'] = y_test_p
    df['Prediction'] = y_pred_p
    df['Forecaster'] = [mr.Rstat2M(mean=i, std=0.01, unit='Jupiter', sample_size=100, grid_size=100)[0] for i in list(X_test_p)]

    df['Resid Test-Prediction'] = np.abs(df['Test']-df['Prediction'])
    df['Resid Test-Forecaster'] = np.abs(df['Test']-df['Forecaster'])
    print(tabulate(df, tablefmt='psql', showindex=False, headers='keys'))
    print('Residual sum for Prediction: ',df['Resid Test-Prediction'].sum(),'\nResidual sum for Forecaster: ', df['Resid Test-Forecaster'].sum())


compare_predict_vs_test(X_test, y_test, y_pred)

+---------+--------------+--------------+-------------------------+-------------------------+
|    Test |   Prediction |   Forecaster |   Resid Test-Prediction |   Resid Test-Forecaster |
|---------+--------------+--------------+-------------------------+-------------------------|
| 0.8     |    0.727009  |   0.218197   |             0.0729913   |             0.581803    |
| 0.812   |    0.805111  |   3.00234    |             0.00688913  |             2.19034     |
| 0.762   |    0.766215  |   5.03887    |             0.00421497  |             4.27687     |
| 0.76    |    0.766287  |   7.65828    |             0.00628697  |             6.89828     |
| 0.02    |    0.0206048 |   0.0177036  |             0.000604828 |             0.00229645  |
| 0.13844 |    0.141585  |   0.143566   |             0.00314513  |             0.00512556  |
| 0.899   |    0.903758  |   2.43534    |             0.00475805  |             1.53634     |
| 0.415   |    0.427259  |   8.54977    |             0.0122

In [342]:
import eli5
from eli5.sklearn import PermutationImportance


# Break off validation set from training data numeric only
X_train, X_test, y_train, y_test = train_test_split(X.select_dtypes('number'), y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=1)

categorical_cols = [cname for cname in X_train.select_dtypes('object').columns 
                        if X_train[cname].nunique() < 10] 

# Select numerical columns
numerical_cols = X_train.select_dtypes('number').columns 

# Preprocessing for numerical data
numerical_transformer =  Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('minmaxscaler', MinMaxScaler(feature_range=(0,1)))
    ])

# Preprocessing for categorical data
categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Run the entire pipeline
permutation_test = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', XGBRegressor(n_estimators=500, random_state=0))
    ])


permutation_test.fit(X_train, y_train)

perm = PermutationImportance(permutation_test, random_state=0).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())


Weight,Feature
1.9305 ± 0.2744,pl_masse
0.0091 ± 0.0365,elat
0.0058 ± 0.0140,sy_gaiamagerr1
0.0002 ± 0.0002,st_masserr1
0.0002 ± 0.0000,pl_orblper
0.0001 ± 0.0002,pl_tranmid
0.0000 ± 0.0000,pl_orbper
0.0000 ± 0.0000,disc_year
0.0000 ± 0.0000,st_dens
0.0000 ± 0.0000,sy_pmra
